#### Libraries

In [1]:
from random import uniform, randint
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#### Entry values, Lists & Variables

In [2]:
#file name?
#file_name = input('File name: ')
file_name = 'Test'

#use standard or custom values?
loop = True
while loop == True:
    #settings = input('Use default settings? (y - yes / n - no')
    settings = 'y'

    #standard values
    if settings == 'y':
        L = 10 #number of cells 
        T = 100 #number of seconds for the simulation
        DENSITY = 0.5 #how many cars
        vmax = 5 #maximum car speed 
        p = 0.25 #probability of random stop
        tp = 0.3 #truck/total ratio
        n_lanes = 1 #number of n_lanes in each direction
        flowList = np.full(2*n_lanes, 900, dtype = "int32") #entry flow in each lane [lane 1, lane 2, lane 3...] in veic/hour
        loop = False

    #custom values
    elif settings == 'n':
        T = int(input('Period of simulation (T): '))
        p = float(input('Probability of random stop (range 0-1): '))
        tp = float(input('Percentage of trucks (range 0-1): '))
        L = round(float(input('Lenght of the simulation in km (L): '))/7.5,0) #NEED TO ROUND UP THE L VALUE
        n_lanes = int(input('Number of lanes in each direction: '))
        vmax = round((float(input('Maximum car speed in km/h (vmax): '))/(3.6*7.5)),0)
        DENSITY = float(input('Initial car density (if you wish to start with an empty road, type 0): '))
        flowList = np.zeros((2*n_lanes), dtype = int)
        lane = 0
        road_side = 0
        while lane <2*n_lanes: 
            if lane < n_lanes:j
                print('Flow in left lane ',lane+1,': ')
            else:
                print('Flow in right lane ',lane+1-n_lanes,': ')
                road_side = 1
            flowList[road_side,int(lane-n_lanes*road_side)] = int(input())
            lane += 1
        loop = False
    
    #in case of error
    else:
        print('Invalid answer! Please use only "y" or "n".')

#### Defining class: Veihcle

In [3]:
class Veic:
    def __init__(self,id,ty,lane,v,x):
        #self.col = color(random(255),random(140),random(255))
        self.id = id #VEIHCLE ID
        self.type = ty #VEIHCLE TYPE, FOLLOWING: 0 = car, 1 = truck 
        self.lane = lane #CURRENT LANE
        self.v = v #SPEED
        self.x = x #REAR BUMPER POSITION

    #DRIVING RULES
    def update(self):
    #STEP 1: acceleration 
    # All cars that have not already reached the maximal velocity vmax acceleration by one unit: v -> v+1         
        if self.v < (vmax-self.type):
            self.v += 1        
        
    #STEP 2: lane change and safety distance
    #If a car has d empty cells in front of it and is its velocity v (after step 1) larger then d, then it reduces the velocity to d: v -> min{d,v}        
        gapList = safe_distances(self)
                    
        if gapList[0] < self.v:
            self.v = gapList[0]
            
    #STEP 3: randomization
    # With probability p, the velocity is reduced by one unit (if v after step 2):v -> v-1
        if self.v > 0:
            if uniform(0,1) < p:
                self.v -= 1
        elif self.v < 0:
            self.v = 0

    #STEP 4: driving
    #After steps 1-3 the new velocity vn for each car n has been determined forward by vn cells: xn -> xn+vn.
        
        #move through list
        sim[self.lane][self.x] = 0
        self.x += self.v
        
        #advancing and leaving the system
        if self.x < L:
            sim[self.lane][self.x] = self
            
    #LANE CHANGE (both inside the veihcle's info and in the simulation)
    def change_lane(self,move):
        sim[self.lane][self.x] = int(0)
        sim[self.lane+move][self.x] = self
        self.lane += move

#### List functions

In [4]:
# def foward(veic):
    
#     lane = int(veic.lane)
#     x = int(veic.x)
#     v = int(veic.v)
    
#     print(type(x),type(v),type(lane))
    
#     sim[lane][x] = int(0)
#     sim[lane][x+v] = veic

# def apply_lane_change(veic,move):
    
#     lane = veic.lane
#     x = int(veic.x)
    
#     sim[lane][x] = int(0)
#     sim[lane+move][x] = self
    
# def remove_veihcle(veic):
    
#     lane = veic.lane
#     x = veic.x
    
#     sim[lane][x] = int(0)

#### Search veihcle with ID

In [5]:
def search_id(id):
    x = 0
    while x < L:
        lane = 0
        while lane < 2*n_lanes:
            if sim[lane][x] != 0:
                if sim[lane][x].id == id:
                    return sim[lane][x]
            lane+=1
        x+=1

#### Gap acceptance rule

In [6]:
#SAFETY DISTANCES
def safe_distances(veic):
    
    #GET CAR DATA
    lane = veic.lane
    x = veic.x
    
    #ADVANCING
    #CHECK SAFETY FRONT DISTANCE
    dist = 0
    while dist < vmax:
        if (x+1+dist) < (L-1):
            if sim[lane][x+1+dist] == 0:
                dist += 1
            else:
                break
        else:
            dist = vmax      

    #CHECK BORDER SAFETY DISTANCES    
    #create gapList to hold border safety distances (front-left, back-left, front-right and back-right)
    dList = np.zeros(3, dtype = "int32")
    dList[0] = dist
    
    #FIND ROAD SHOULDERS
    if lane < n_lanes:
        left_shoulder = 0
        right_shoulder = n_lanes-1
    else:
        left_shoulder = n_lanes
        right_shoulder = 2*n_lanes-1

    #SINGLE LANE
    if n_lanes == 1:
        dList[1] = -1 #LEFT
        dList[2] = -1 #RIGHT
        
    #TWO OR MORE LANES    
    elif n_lanes == 2:
        
        #check if RIGHT or LEFT
        if lane == left_shoulder: 
            dList[1] = -1 #LEFT
            dList[2] = gap(sim[lane][x],1) #RIGHT

        elif lane == right_shoulder:
            dList[1] = gap(sim[lane][x],-1) #LEFT
            dList[2] = -1 #RIGHT
            
        else: 
            dList[1] = gap(sim[lane][x],-1) #LEFT
            dList[2] = gap(sim[lane][x],1) #RIGHT
            
    return dList

#### Driver perception rule (left-right turns)

In [7]:
def gap(veic, move):
    
    #GET CAR DATA
    lane = veic.lane
    x = veic.x
    
    #FIND DIST 
    #finds distance to desired neighbor in the new lane by growing "dist" till it finds an occupied space  
    dist = 0
    while dist < vmax:
        if x+dist < L:
            if sim[lane+move][x+dist] == 0:
                dist += 1
            else:
                break
        else:
            break

    #SIZE? #reduce the distance in 1 in case it's a truck
    dist = dist - veic.type

    return dist

#### Next veihcle in line for the given position [lane,x]

In [8]:
def closest(lane,x):
    
    while sim[lane][x] == 0:
        x += 1
        if x > (L-1):
            return vmax
    
    return sim[lane][x]

#### Lane Change Rule

In [9]:
def try_lane_change(veihcle,move):
    
    #GET RELEVANT VEIHCLE INFO
    lane = int(veihcle.lane)
    x = int(veihcle.x)
    gapList = safe_distances(veihcle)
    if move == 1:
        gap = 2
    else:
        gap = 1
    
    #REQUIREMENTS FOR LANE CHANGE: 
    # the next driver in the desired lane is faster then the current next driver (it is desired to change lanes).
    # there is room for the changing manouver (gap check).
    
    if gapList[gap] > 0:
        
        try:
            new_next_speed = closest(lane+move,x).v
        except:
            new_next_speed = closest(lane+move,x)

        try:
            next_speed = closest(lane+move,x).v
        except:
            next_speed = closest(lane+move,x)
    
        if new_next_speed > next_speed:
            return True
        else:
            return False
    else:
        return False

#### Simple display

In [10]:
def simple_display():
    
    ID_list = np.full_like(sim,'_')
    type_list = np.full_like(sim,'_')
    speed_list = np.full_like(sim,'_')
    
    lane = 0
    while lane < 2*n_lanes:
        x = 0
        while x < L:
            if sim[lane][x] != 0:
                ID_list[lane][x] = sim[lane][x].id
                type_list[lane][x] = sim[lane][x].type
                speed_list[lane][x] = sim[lane][x].v
            x += 1
        lane += 1
        
    #print("ID_list:\n",ID_list,"\n")
    #print("type_list:\n",type_list,"\n")
    print("speed_list:\n",speed_list,"\n")

#### SIMULATION SETUP

In [23]:
print("Simulation setup...\n")

#number of veihcles in the begining of the simulation (trucks counts as 2)
n0 = int(DENSITY*L*(1-tp))
ID = 0

#SET LISTS
sim = np.zeros([2*n_lanes,L],dtype = object) #veic[lane][ID]
detector = np.zeros([2*n_lanes,2],dtype = 'int32') #sensor[lane number][i] with [0] for entrance and [1] for exit
detector[:,0] = n0 #count cars already in the simulation

print("detectors\n",detector,"\n")

#CREATE POSMATRIX

# define unique positions to place cars (veihcles can't share the same space)
posMatrix = np.full([2*n_lanes,max(1,n0)],-1,dtype = 'int32')

lane = 0
while lane < 2*n_lanes:
    i = 0
    while i < n0:
        if posMatrix[lane][i] == -1:
            position = randint(0,L-1)
            if position not in posMatrix[lane]:
                posMatrix[lane][i] = position
                i += 1
    lane += 1

if n0 > 0:
    posMatrix = np.sort(posMatrix)
    print("posMatrix:\n",posMatrix,"\n")
        
    #PLACE VEIHCLES IN DECRECENT ID ORDER (car "0" is always the first to lead)
    # { >(id_9)> ... >(id_6)> ... >(id_3)> ... >(id_1)> ... ...}
    # { >(id_10)> >(id_8)> ... ... >(id_4)> ... ... ... >(id_0)>}
    # { >(id_11)> ... >(id_7)> ... >(id_5)> ... >(id_2)> ... ...}

    #INSERT CARS
    x = (L-1)
    while x > -1:
        lane = 0
        while lane < 2*n_lanes:
            if x in posMatrix[lane]:
                sim[lane][x] = Veic(ID,0,lane,randint(0,vmax+1),x) #OBS. testar se vmax melhor que randint(0,vmax)
                ID += 1
            lane += 1
        x -= 1

    #INSERT TRUCKS
    trucks = int(DENSITY*L*tp)
    print((n0-trucks)*2*n_lanes,"cars and",trucks*2*n_lanes,"trucks", "placed\n")

    while trucks > -1:
        lane = randint(0,2*n_lanes-1)
        i = randint(0,min(1,n0-2))
        if (posMatrix[lane][i]+1 not in posMatrix[lane]) and (sim[lane][posMatrix[lane][i]].type == 0):
            sim[lane][posMatrix[lane][i]].type = 1
            trucks -=1
    
simple_display()

print("All set!")

Simulation setup...

detectors
 [[3 0]
 [3 0]] 

posMatrix:
 [[2 3 8]
 [0 5 8]] 

4 cars and 2 trucks placed

speed_list:
 [['_' '_' 2 4 '_' '_' '_' '_' 5 '_']
 [2 '_' '_' '_' '_' 3 '_' '_' 6 '_']] 

All set!


#### File handling

In [19]:
sim_file = file_name+'.csv'
data_file = file_name+'_dat.csv'
diagram_file = []
for lane in range(2*n_lanes):
    diagram_file.append(file_name + '_0' + str(lane) + '.txt')

#OPEN FILES
try:
    sim_hand = open(sim_file,'w')
except:
    print('File cannot be opened: ',sim_file)
    exit()    

sim_hand.write('t,ID,type,lane,pos,speed\n')

25

#### RUN SIMULATION

In [24]:
t = 0
print('Starting...\n')

while t < T:
    print('t: ',t)
    simple_display()

    #LANE CHANGE PHASE
    #changing lanes BEFORE moving foward to avoid accidents.
    
    # allow LEFT movements in even time-steps and RIGHT movements in odd time-steps
    if t%2 == 0:
        move = -1
    else:
        move = 1
    
    # set lane change
    x = (L-1)
    while x > -1:
        lane = 0
        while lane < 2*n_lanes:
            if sim[lane][x] != 0:
                if try_lane_change(sim[lane][x],move) == True:
                    sim[lane][x].change_lane(move)
            lane += 1
        x -= 1
    
    #ADVANCE PHASE
    #update cars from Upstream to Downstream.
    x = (L-1)
    while x > -1:
        lane = 0
        while lane < 2*n_lanes:
            if sim[lane][x] != 0:
                sim[lane][x].update()
            lane += 1
        x -= 1
    
    #REGISTER PHASE
    #save current veihcle disposition
    x = (L-1)
    while x > -1:
        lane = 0
        while lane < 2*n_lanes:
            if sim[lane][x] != 0:
                sim_hand.write(str(t)+','+str(sim[lane][x].id)+','+str(sim[lane][x].type)+','+str(lane)+','+str(x)+','+str(sim[lane][x].v)+'\n')
            lane += 1
        x -= 1
    
    #ENTRY FLOW
    lane = 0
    while lane < 2*n_lanes:
            
        #check if it's time to add a new veic and if there is space for such
        if (t*flowList[lane]/3600-detector[lane][0]) > 1 and sim[lane][0] == 0:
                
            #roll type of new veihcle
            ty = 0
            if uniform(0,1) < tp:
                ty = 1
            
            #add veihcle to position
            ID += 1
            sim[lane][0] = Veic(ID,ty,lane,randint(1,vmax),0)
            detector[lane][0] += 1
            
        lane += 1


    #COLECTING DATA FROM THE TIME-STEP  (intended for the "_dat" file)
    #density.append(n/L)
    
    #NEXT TIME STEP
    t += 1

print('Done!')

Starting...

t:  0
speed_list:
 [['_' '_' 2 4 '_' '_' '_' '_' 5 '_']
 [2 '_' '_' '_' '_' 3 '_' '_' 6 '_']] 

t:  1
speed_list:
 [['_' '_' '_' '_' 2 '_' '_' 4 '_' '_']
 ['_' '_' 2 '_' '_' '_' '_' '_' '_' 4]] 

t:  2
speed_list:
 [['_' '_' '_' '_' '_' '_' 2 '_' '_' '_']
 ['_' '_' '_' '_' 2 '_' '_' '_' '_' '_']] 

t:  3
speed_list:
 [['_' '_' '_' '_' '_' '_' '_' '_' 2 '_']
 ['_' '_' '_' '_' '_' '_' '_' 3 '_' '_']] 

t:  4
speed_list:
 [['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']] 

t:  5
speed_list:
 [['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']] 

t:  6
speed_list:
 [['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']] 

t:  7
speed_list:
 [['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']] 

t:  8
speed_list:
 [['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']] 

t:  9
speed_list:
 [['_' '_' '_' '_' '_' '_

#### Generate Simulation Table

In [21]:
#GRAPHS
df1 = pd.read_csv(file, sep=',') 

df1.head(-1)

FileNotFoundError: [Errno 2] File b'Test' does not exist: b'Test'

#### SIMULATION REPORT

In [ ]:
#0PEN DATA FILE
# try:
#     data_hand = open(data_file,'w')
# except:
#     print('File cannot be opened: ',data_file)
#     exit()  

# data_hand.write('t,k,\n')

# t = 0
# while t < T:
#     data_hand.write(str(t)+','+str(density[t])+'\n')
#     t += 1   
    
# data_hand.close() 

#### SPACE-TIME DIAGRAMS